## Use this starter notebook to run some new experiments

In [1]:
# Change path as needed
MAP_PATH = "../tests/pickle_data/GresvanAIE.xz"

In [2]:
# load cell magic things
%load_ext line_profiler
%load_ext Cython

In [3]:
# imports
import matplotlib.pyplot as plt
import numpy as np

from sc2.ids.unit_typeid import UnitTypeId
from sc2.bot_ai import BotAI
from sc2.position import Point2
from sc2.dicts.unit_trained_from import UNIT_TRAINED_FROM
from sc2.game_data import Race
from sc2.unit import Unit
from sc2.units import Units

from tests.load_bot_from_pickle import get_map_specific_bot

In [4]:
# setup a burnysc2 BOTAI instance we can test with
bot: BotAI = get_map_specific_bot(MAP_PATH)

In [12]:
# sanity check that the bot instance is working
plt.imshow(bot.game_info.pathing_grid.data_numpy)

NameError: name 'bot' is not defined

In [43]:
from sc2.constants import CREATION_ABILITY_FIX
from sc2.ids.ability_id import AbilityId
from sc2.ids.unit_typeid import UnitTypeId
from collections import Counter
from sc2.data import Race
from sc2.unit import Unit

def _abilities_count_and_build_progress(self):
    """Cache for the already_pending function, includes protoss units warping in,
    all units in production and all structures, and all morphs"""
    abilities_amount: Counter[AbilityId] = Counter()
    unit: Unit
    for unit in self.units + self.structures:
        for order in unit.orders:
            abilities_amount[order.ability.exact_id] += 1
        if not unit.is_ready and (self.race != Race.Terran or not unit.is_structure):
            # If an SCV is constructing a building, already_pending would count this structure twice
            # (once from the SCV order, and once from "not structure.is_ready")
            if unit.type_id in CREATION_ABILITY_FIX:
                if unit.type_id == UnitTypeId.ARCHON:
                    # Hotfix for archons in morph state
                    creation_ability = AbilityId.ARCHON_WARP_TARGET
                    abilities_amount[creation_ability] += 2
                else:
                    # Hotfix for rich geysirs
                    creation_ability = CREATION_ABILITY_FIX[unit.type_id]
                    abilities_amount[creation_ability] += 1
            else:
                creation_ability: AbilityId = self.game_data.units[unit.type_id.value].creation_ability.exact_id
                abilities_amount[creation_ability] += 1

    return abilities_amount

In [44]:
%timeit _abilities_count_and_build_progress(bot)

23.2 μs ± 1 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
%%cython
from libc.stdlib cimport malloc, free
from libc.string cimport memset
from cython cimport boundscheck, wraparound
from cython_extensions.ability_mapping cimport map_value
from cython_extensions.ability_mapping cimport STRUCT_ABILITIES
from sc2.data import Race


cdef struct AbilityCount:
    int ability_id
    int count
# Disable Python checks for speed
@boundscheck(False)
@wraparound(False)
cpdef AbilityCount[:] abilities_count_structures(object bot):
    """
    Build a C array indexed by ability_id that stores counts.
    Returns: memoryview of AbilityCount (size = 2200)
    """

    cdef int MAX_ABILITIES = 2200
    cdef AbilityCount* arr = <AbilityCount*> malloc(MAX_ABILITIES * sizeof(AbilityCount))

    # ... fill arr ...

    memset(arr, 0, MAX_ABILITIES * sizeof(AbilityCount))

    cdef object unit
    cdef object order
    cdef int aid
    # special ability ids that should always be counted if seen

    cdef object structures = bot.structures
    cdef object workers = bot.workers


    # Workers orders → ability_id count

    
    for unit in workers:
        for order in unit.orders:
            aid = <int> order.ability.exact_id.value #FUTURE add mapping table for order.ability?
            if 0 <= aid < MAX_ABILITIES:
                arr[aid].count += 1

    # Structures → build progress < 1.0 → increment creation ability
    if bot.race != Race.Terran:
        for unit in structures:
            if <double> unit.build_progress < 1.0:
                aid = <int> map_value(unit.type_id.value)
                if aid!=-1:
                    arr[aid].count += 1
    
    #for terran, count PF, OC, Reactor, Tech Lab too
    else:
        for unit in structures:
            aid = <int> map_value(unit.type_id.value)
            if <double> unit.build_progress < 1.0:
                if STRUCT_ABILITIES[aid]:
                    arr[aid].count += 1
            elif aid == 318: # Command Center for OC and PF
                for order in unit.orders:
                    aid = <int> order.ability.exact_id.value
                    if STRUCT_ABILITIES[aid]:
                        arr[aid].count += 1
        
    # Return as Python-usable memoryview
    return <AbilityCount[:MAX_ABILITIES]> arr

Content of stdout:
_cython_magic_11f541c904ef2664b63b1eb0afd072c85eabb86d936e6690a88703f18e21a9c6.c
   Bibliothek "C:\Users\jonas\.ipython\cython\Users\jonas\.ipython\cython\_cython_magic_11f541c904ef2664b63b1eb0afd072c85eabb86d936e6690a88703f18e21a9c6.cp313-win_amd64.lib" und Objekt "C:\Users\jonas\.ipython\cython\Users\jonas\.ipython\cython\_cython_magic_11f541c904ef2664b63b1eb0afd072c85eabb86d936e6690a88703f18e21a9c6.cp313-win_amd64.exp" werden erstellt.
Code wird generiert.
Codegenerierung ist abgeschlossen.

TypeError: C variable cython_extensions.ability_mapping.STRUCT_ABILITIES has wrong signature (expected int [0x654], got int [1620])

In [10]:
%timeit abilities_count_structures(bot)

20.9 μs ± 601 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [30]:
from cython_extensions import cy_structure_pending


In [ ]:
%timeit -n 10000 cy_structure_pending(bot, UnitTypeId.BARRACKS)

16.4 μs ± 913 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [35]:
%timeit -n 10000 bot.already_pending(UnitTypeId.BARRACKS)

2.22 μs ± 156 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
%%cython
from cython cimport boundscheck, wraparound
from cython_extensions.ability_mapping cimport map_value
from cython_extensions.ability_order_tracker import cy_abilities_count_structures

cdef struct AbilityCount:
    int ability_id
    int count

@boundscheck(False)
@wraparound(False)
cpdef unsigned int cy_structure_pending_func(
        object bot,
        object structure_type,
    ):
    cdef:
        unsigned int num_pending = 0
        object counts_and_progress
        object creation_ability = None
        object local_get
        int ability_int
        object ability_obj
        int count
        int target = <int> structure_type.value
        cdef AbilityCount item

    # Use optimized Cython function to get ability counts

    counts_and_progress = cy_abilities_count_structures(bot) #returns a c array memoryview


    # local_get reduces repeated attribute accesses
    arr_len = counts_and_progress.shape[0]
    target_created_ability = <int> map_value(target)
    #print("target_created_ability:", target_created_ability, "struct:",  structure_type)
    if 0 <= target_created_ability < arr_len:
        item = counts_and_progress[target_created_ability]
        num_pending += item.count
    return num_pending

Content of stdout:
_cython_magic_0e5510fb85a14ca481749f6a99678b795871259d5387a0e4521b59e64c375705.c
   Bibliothek "C:\Users\jonas\.ipython\cython\Users\jonas\.ipython\cython\_cython_magic_0e5510fb85a14ca481749f6a99678b795871259d5387a0e4521b59e64c375705.cp313-win_amd64.lib" und Objekt "C:\Users\jonas\.ipython\cython\Users\jonas\.ipython\cython\_cython_magic_0e5510fb85a14ca481749f6a99678b795871259d5387a0e4521b59e64c375705.cp313-win_amd64.exp" werden erstellt.
Code wird generiert.
Codegenerierung ist abgeschlossen.

In [40]:
%timeit cy_structure_pending_func(bot, UnitTypeId.BARRACKS)

870 ns ± 6.01 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [6]:
worker=bot.workers.first

In [7]:
%timeit worker.orders[0].ability.exact_id

570 ns ± 3.35 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [8]:
%timeit worker.orders[0].ability._proto.ability_id

154 ns ± 0.625 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
